# Psi4GUI

A (Jupyter/voila) interface to Psi4 electronic structure program, with MO rendering using <s>NGLview</s> pymol3D

In [13]:
import psi4
psi4.core.set_output_file('output.log', True)
import numpy as np
import matplotlib.pyplot as plt
import py3Dmol
import nglview as nv
import glob
import os
import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display

elements =          [None,
          'H' ,                                                                                'He',
          'Li','Be',                                                  'B' ,'C' ,'N' ,'O' ,'F' ,'Ne',
          'Na','Mg',                                                  'Al','Si','P' ,'S' ,'Cl','Ar',
          'K' ,'Ca','Sc','Ti','V' ,'Cr','Mn','Fe','Co','Ni','Cu','Zn','Ga','Ge','As','Se','Br','Kr',
          'Rb','Sr','Y' ,'Zr','Nb','Mo','Tc','Ru','Rh','Pd','Ag','Cd','In','Sn','Sb','Te','I' ,'Xe',
          'Cs','Ba','La', #Lantanides:  
#                  ---------------------------------------------------
                    'Ce','Pr','Nd','Pm','Sm','Eu','Gd',#
                    'Tb','Dy','Ho','Er','Tm','Yb','Lu',#
#                  ---------------------------------------------------
                         'Hf','Ta','W' ,'Re','Os','Ir','Pt','Au','Hg','Tl','Pb','Bi','Po','At','Rn',
         'Fr','Ra','Ac', #Actinides:
#                  ---------------------------------------------------
                   'Th','Pa','U' ,'Np','Pu','Am','Cm',#
                   'Bk','Cf','Es','Fm','Md','No','Lr'#
#                  ---------------------------------------------------
         ]

def cubetoxyz(cubefile):
    with open(cubefile) as f:
        atoms=[]
        line = f.readline()
        data = line.split()
        while len(data) != 6:
            line = f.readline()
            data = line.split()
            if len(data)==5:
                atoms.append(data)
    newxyz = str(len(atoms))+'\n'
    newxyz +='New xyz\n'
    for atom in atoms:
        xyz = [ '{:8.3f}'.format(float(x)*0.529177) for x in atom[2:]]
        newxyz += elements[int(atom[0])]+' '+' '.join(xyz)+'\n'
        
    return newxyz

def cubetopdb(cubefile):
    with open(cubefile) as f:
        atoms=[]
        line = f.readline()
        data = line.split()
        while len(data) != 6:
            line = f.readline()
            data = line.split()
            if len(data)==5:
                atoms.append(data)
    newpdb = 'TITLE pdb geometry'
    for i,atom in enumerate(atoms):
        atname=elements[int(atom[0])]
        xyz = [ float(x)*0.529177 for x in atom[2:]]
        line = "{:<6}{:>5d} {:<4}{:1}{:>3} {:1}{:>4d}{:1}   {:>8.3f}{:>8.3f}{:>8.3f}{:22}{:>2}".\
        format('HETATM',i+1,atname,' ','RES',' ',1,' ',*xyz,' ',atname)
        newpdb += "\n"+line
    return newpdb

def print_mo(mo,iso=0.02):
    # Search cubefile
    cubefiles=glob.glob('Psi_a_'+str(mo)+'_*')
    if len(cubefiles) > 1:
        print('Too many cubefiles for MO '+str(mo))
        return None
    if len(cubefiles) == 0:
        print('No cuefile for MO '+str(mo))
        return None
    cubefile=cubefiles[0]
    MO = open(cubefile).read()
    # xyz geom
    geomxyz = cubetoxyz(cubefile)
    view = py3Dmol.view()
    view.addModel(geomxyz,'xyz')
    view.setStyle({'sphere':{'scale':0.25}})
    view.addStyle({'stick':{'radius':0.1}})
    view.addVolumetricData(MO, "cube", {'isoval':  iso, 'color': "blue", 'opacity': 0.75})
    view.addVolumetricData(MO, "cube", {'isoval': -iso, 'color': "red", 'opacity': 0.75})
    view.rotate(0, {'x':1,'y':1,'z':1})
    view.zoomTo()
    
    return view, cubefile


def print_mo_geom(mo,geom,iso=0.02):
    # Search cubefile
    folder='GEOM_{:03g}'.format(geom)+'/'
    cubefiles=glob.glob(folder+'Psi_a_'+str(mo)+'_*')
    if len(cubefiles) > 1:
        print('Too many cubefiles for MO '+str(mo))
        return None
    if len(cubefiles) == 0:
        print('No cuefile for MO '+str(mo)+' at geom '+str(geom))
        return None
    cubefile=cubefiles[0]
    MO = open(cubefile).read()
    # xyz geom
    geomxyz = cubetoxyz(cubefile)
    view = py3Dmol.view()
    view.addModel(geomxyz,'xyz')
    view.setStyle({'sphere':{'scale':0.25}})
    view.addStyle({'stick':{'radius':0.1}})
    view.addVolumetricData(MO, "cube", {'isoval':  iso, 'color': "blue", 'opacity': 0.75})
    view.addVolumetricData(MO, "cube", {'isoval': -iso, 'color': "red", 'opacity': 0.75})
    view.rotate(0, {'x':1,'y':1,'z':1})
    view.zoomTo()
    
    return view, cubefile

def print_mo_geom_ngl(mo,geom,iso=0.02):
    # Search cubefile
    folder='GEOM_{:03g}'.format(geom)+'/'
    cubefiles=glob.glob(folder+'Psi_a_'+str(mo)+'_*')
    if len(cubefiles) > 1:
        print('Too many cubefiles for MO '+str(mo))
        return None
    if len(cubefiles) == 0:
        print('No cuefile for MO '+str(mo)+' at geom '+str(geom))
        return None
    cubefile=cubefiles[0]

    geofile='geom_tmp.pdb'
    with open(geofile,'w') as f:
        print(cubetopdb(cubefile),file=f)
    view = nv.show_file(geofile)
    #view = nv.NGLWidget()
    view.add_component(cubefile)
    view.component_1.clear()
    view.component_1.add_surface(opacity=0.5, color='blue', isolevel=1., probe_radius=10.)
    view.component_1.add_surface(opacity=0.5, color='red', isolevel=-1., depthWrite=False)
    view.parameters = {
        "clipNear": 0, "clipFar": 100, "clipDist": 1
    }
    
    return view, cubefile


def xyzfile2geom(fname,charge=None,mult=None,symm=None):
    '''Extact xyz geometry from xyz file (discarding 2 first lines)
    '''
    with open(fname) as fxyz:
        geomxyz = fxyz.read()
    # We can provide the geom without 2 first lines of xyz file. 
    # This allows additional input (such as units -ang by default- or symmetry)
    geo = geomxyz.split('\n')
    geo = '\n'.join(geo[2:])
    
    if charge is not None and mult is not None:
        geo = str(charge)+' '+str(mult)+'\n'+geo
        
    if symm:
        geo += '\nsymmetry '+symm
    
    return geo

def xyzstr2geom(geomxyz,charge=None,mult=None,symm=None):
    '''Extact xyz geometry from xyz file (discarding 2 first lines)
    '''
    # We can provide the geom without 2 first lines of xyz file. 
    # This allows additional input (such as units -ang by default- or symmetry)
    geo = geomxyz.split('\n')
    geo = '\n'.join(geo[2:])
    
    if charge is not None and mult is not None:
        geo = str(charge)+' '+str(mult)+'\n'+geo
        
    if symm:
        geo += '\nsymmetry '+symm
    
    return geo


# Input geometry and options

In [78]:
import os
from ipyfilechooser import FileChooser

# Load local file
path=os.getcwd()

# Create and display a FileChooser widget
fc = FileChooser(path)
fc.title = '<b>Local File</b>'
display(fc)

from ipywidgets import FileUpload

# Upload file
upload = FileUpload()
text = widgets.Output()
with text:
    print('Remote File')
display(text,upload)

def on_button_clicked(b):
    global mol
    output1.clear_output()
    if upload.get_state()['_counter']>0:
        with output1:
            print('Using remote file: ',upload.metadata[0]['name'])
        ftype = upload.metadata[0]['name'].split('.')[-1]
        filedata = upload.data[0].decode()
    elif fc.get_interact_value():
        with output1:
            print('Using local file: ',fc.selected_filename)
        ftype = fc.selected_filename.split('.')[-1]
        with open(fc.selected) as f:
            filedata = f.read()
    else:
        with output1:
            print('No file selected')
        return None
    if ftype != 'xyz':
        raise BaseException('Unsupported file type: '+ftype)
        
    # Build geom string
    geom = xyzstr2geom(filedata,charge=0,mult=1)
    
    # Show geom
    with output1:
        print('Coordinates of your molecule:\n')
        print(geom)
    
    # Build psi4 molecule object
    mol = psi4.geometry(geom)


# Load geom
button = widgets.Button(description="Load geometry")
output1 = widgets.Output()

display(button, output1)   

button.on_click(on_button_clicked)

FileChooser(path='/home/cerezo/Documentos/DOCENCIA/2020-21/QuiFiIII/UsingPsi4Jupyter', filename='', title='HTM…

Output()

FileUpload(value={}, description='Upload')

Button(description='Load geometry', style=ButtonStyle())

Output()

# Psi4 calculation

In [46]:
button = widgets.Button(description="Single point energy")
output2 = widgets.Output()

display(button)

def on_button_clicked(b):
    global E, wfn
    output2.clear_output()
    with output2:
        print('Running energy calc...')
    E, wfn = psi4.energy('b3lyp/6-31G(d)',return_wfn=True)
    output2.clear_output()
    with output2:
        print('Energy (a.u.) = ',E)
        print("Calculation finished")    

button.on_click(on_button_clicked)



Button(description='Single point energy', style=ButtonStyle())

In [45]:
button = widgets.Button(description="Optimize")
#output3 = widgets.Output()

display(button)

def on_button_clicked(b):
    global E, wfn, mol
    output2.clear_output()
    with output2:
        print('Running optimization...')
    E, wfn = psi4.optimize('b3lyp/6-31G(d)',return_wfn=True)
    # Compute bond distance
    geom = mol.geometry().np * 0.529177
    np.linalg.norm(geom[1]-geom[0])
    output2.clear_output()
    with output2:
        print('Optimized geometry:')
        print(mol.to_string('xyz'))
        print('Optimized distance')
        print('Energy (a.u.) = ',E)
        print("Calculation finished")    

button.on_click(on_button_clicked)

Button(description='Optimize', style=ButtonStyle())

In [42]:
display(output2)

Output()

# Molecular orbitals

In [38]:
button = widgets.Button(description="Compute MO")
output4 = widgets.Output()

display(button, output4)


def on_button_clicked(b):
    global wfn, mo_ener
    # Compute cube files
    %rm *cube
    psi4.driver.p4util.cubeprop(wfn)
    
    # MO energies
    # Get MO energies. Per irrep 
    mo_ener_symm = wfn.epsilon_a().nph
    # Concatenate energies for all irrep in a single array
    mo_ener=np.concatenate(mo_ener_symm)
    # And sort by energy
    mo_ener.sort()
    # Show
    #mo_ener
    
    with output4:
        print("MO done")    
    


button.on_click(on_button_clicked)


Button(description='Compute MO', style=ButtonStyle())

Output()

In [39]:
def display_mo(mo):
    i=mo
    try:
        view, cubefile = print_mo(i,iso=0.05)
        mo_name = cubefile.replace('.cube','')
        mo_name = mo_name.split('/')[-1]
        sym_label = mo_name.split('_')[-1]
        print('MO: {:<4}   E={:8.3f}     SymmLabel: {:4}'.format(i,mo_ener[i-1],sym_label))
        view.setPerceivedDistance(10)
        view.rotate(90, {'x':1,'y':1,'z':1})
        return view.show()
    except:
        return None

#nmo = len(mo_ener)
interactive_plot = interactive(display_mo, mo=(1,10,1))
out = interactive_plot.children[-1]
out.layout.height = '500px'
interactive_plot

interactive(children=(IntSlider(value=5, description='mo', max=10, min=1), Output(layout=Layout(height='500px'…